In [4]:
import pandas as pd
import utils
import numpy as np
import matplotlib.pyplot as plt
import time as timefn
import scipy.io
import csv
from bs4 import BeautifulSoup
import json
import os
import datetime

# Downloading Data from Original Sources

## Go to Loading_Data_and_Defining_subtasks first!

This notebook shows where the original data was downloaded from (if need to download more, etc.)

## Weather Observation data

Obtained from NOAA site, have to fill out request form and search

Start here: https://www.ncdc.noaa.gov/cdo-web/datasets

The data I used is Local Climitological Data and Hourly Normals

## Weather Forecast data

Downloaded from DarkSky API : https://darksky.net/dev

In [ ]:
#Weather Forecast data

#Lets look at the forecast every day at noon to decide what to do the next day (starting at midnight)

DarkSky_key = '[key]' #Use your own key (register at https://darksky.net/dev), get 1000 calls/day
#Santa Maria, CA
time_forecast = 1477915200 #UNIX epoch time
d = []
for ii in range(365):
      
    URL_json = 'https://api.darksky.net/forecast/' + DarkSky_key + '/34.9530,-120.4357,' + str(time_forecast) + '?exclude=currently,minutely,daily,alerts,flags'
    forecast_filename = utils.get_data(URL_json,'json')
    d.append({'time': time_forecast, 'Name': forecast_filename})
    time_forecast = time_forecast + 3600*24

df_SMforecast_filenames = pd.DataFrame(d)

df_SMforecast_filenames.to_csv('SM_forecast_filenames.csv')

#Salem, OR
time_forecast = 1477915200 #UNIX epoch time
d = []
for ii in range(365):
      
    URL_json = 'https://api.darksky.net/forecast/' + DarkSky_key + '/44.9429,-123.0351,' + str(time_forecast) + '?exclude=currently,minutely,daily,alerts,flags'
    forecast_filename = utils.get_data(URL_json,'json')
    d.append({'time': time_forecast, 'Name': forecast_filename})
    time_forecast = time_forecast + 3600*24

df_Salemforecast_filenames = pd.DataFrame(d)

df_Salemforecast_filenames.to_csv('Salem_forecast_filenames.csv')

In [3]:
#Fairbanks, AK
DarkSky_key = '0350bd35c145b65f9c56808782d112a4' #Use your own key (register at https://darksky.net/dev), get 1000 calls/day

time_forecast = 1451649600 #UNIX epoch time
d = []
for ii in range(365):
      
    URL_json = 'https://api.darksky.net/forecast/' + DarkSky_key + '/64.8378,-147.7164,' + str(time_forecast) + '?exclude=currently,minutely,daily,alerts,flags'
    forecast_filename = utils.get_data(URL_json,'json')
    d.append({'time': time_forecast, 'Name': forecast_filename})
    time_forecast = time_forecast + 3600*24

df_AKforecast_filenames = pd.DataFrame(d)

df_AKforecast_filenames.to_csv('SM_forecast_filenames.csv')

In [11]:
forecasts_json = []
for file in df_AKforecast_filenames['Name']:
    with open(file, "r") as f:
        forecasts_json.append(json.load(f))

In [13]:
columns = ("latitude", 
           "longitude", 
           "time", 
           "summary", 
           "precipIntensity",
           "precipProbability",
           "temperature",
           "apparentTemperature",
           "dewPoint",
           "humidity",
           "pressure",
           "windSpeed",
           "windBearing",
           "cloudCover",
           "uvIndex",
           "visibility"
           )

keys = columns

end = len(keys)

#%%

with open("AK_forecast.csv", "w+") as out:
    
    c = 0
    
    # Header row
    for key in keys:
        c += 1
        out.write(key)
        
        if c != end:
            out.write(",")
        
    for forecast in forecasts_json:
        
        latitude = forecast["latitude"]
        longitude = forecast["longitude"]
        
        for hour in forecast["hourly"]["data"]:
            
            out.write("\n")
            c = 0
            
            for key in keys:
                
                c += 1
                
                if key == "latitude":
                    out.write(str(latitude))
                    
                elif key == "longitude":
                    out.write(str(longitude))
                
                elif key == "time":
                    time = datetime.datetime.fromtimestamp(int(hour["time"])).strftime('%Y-%m-%d %H:%M:%S')
                    out.write(time)
                
                elif key in hour:
                    out.write(str(hour[key]))
                    
                else:
                    out.write("NaN")
                    
                if c != end:
                        out.write(",")


## Energy Price Data

Downloaded from CA ISO page (server takes forever)

Check out node map : http://www.caiso.com/pricemap/Pages/default.aspx

In [ ]:
#This section is for downloading Energy Price Data

# FOOTHILL_1_N001 - corresponds to San Luis Ob 
#(Note that I tried multiple nodes in Santa Maria, and they each had 1 or 2 months
#with bad data)
start_times = ['20161101','20161201','20161231','20170130','20170227','20170326','20170425','20170524','20170623','20170722','20170821','20170920','20171019']
end_times = ['20161201','20161231','20170130','20170227','20170326','20170425','20170524','20170623','20170722','20170821','20170920','20171019','20171101']

#Can only request data for 30 days at a time
xml_fns = []
for j in range(2,13):
    URL = 'http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&startdatetime=' + start_times[j] + 'T07:00-0000&enddatetime=' + end_times[j] + 'T07:00-0000&version=1&market_run_id=DAM&node=FOOTHILL_1_N001'
    fn = utils.get_data(URL,'zip')
    xml_fns.append(fn)


In [2]:

Node_location = 'HLCR_LNODEWR1' #Salem, OR
start_times = ['20151231','20160130','20160227','20160326','20160425','20160524','20160623','20160722','20160821','20160920','20161019','20161101','20161201']
end_times = ['20160130','20160227','20160326','20160425','20160524','20160623','20160722','20160821','20160920','20161019','20161101','20161201','20161231']

#Can only request data for 30 days at a time
xml_fns2 = []
for j in range(13):
    URL = 'http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&startdatetime=' + start_times[j] + 'T07:00-0000&enddatetime=' + end_times[j] + 'T07:00-0000&version=1&market_run_id=DAM&node=' + Node_location
    fn = utils.get_data(URL,'zip')
    xml_fns2.append(fn)



Extracting zip
Extracting zip
Extracting zip
Extracting zip
Extracting zip
Extracting zip
Extracting zip
Extracting zip
Extracting zip
Extracting zip
Extracting zip
Extracting zip
Extracting zip


## Energy Demand Data

Downloaded from CA ISO page 

In [ ]:
#Download Energy Demand data
start_times = ['20161101','20161201','20161231','20170130','20170227','20170326','20170425','20170524','20170623','20170722','20170821','20170920','20171019']
end_times = ['20161201','20161231','20170130','20170227','20170326','20170425','20170524','20170623','20170722','20170821','20170920','20171019','20171101']

#Can only request data for 30 days at a time
xml_fn_demand = []
for j in range(13):
    URL = 'http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&startdatetime=' + start_times[j] + 'T07:00-0000&enddatetime=' + end_times[j] + 'T07:00-0000&version=1&market_run_id=ACTUAL'
    fn = utils.get_data(URL,'zip')
    xml_fn_demand.append(fn)
    timefn.sleep(5) #so you don't get denied access

## Solar Data

NREL provides solar irradiance data that is very useful to predict PV power output. 

Check out the map: https://maps.nrel.gov/nsrdb-viewer/?aL=UdPEX9%255Bv%255D%3Dt%268VWYIh%255Bv%255D%3Dt%268VWYIh%255Bd%255D%3D1&bL=clight&cE=0&lR=0&mC=40.59727063442024%2C-118.43261718749999&zL=5

and the database for temporal data at many sites: http://rredc.nrel.gov/solar/old_data/nsrdb/1991-2010/

In [ ]:
#Download Solar Data 
df_SM_solar = pd.read_csv('http://rredc.nrel.gov/solar/old_data/nsrdb/1991-2010/data/hourly/723940/723940_2010_solar.csv')
df_Salem_solar = pd.read_csv('http://rredc.nrel.gov/solar/old_data/nsrdb/1991-2010/data/hourly/726940/726940_2010_solar.csv')

## Battery Cycling Data

In [3]:
#Load battery dataset

URL = 'http://ti.arc.nasa.gov/c/5/'

fn = utils.get_data(URL,'zip')

KeyboardInterrupt: 